In [1]:
import hoomd
import hoomd.md

In [2]:
hoomd.context.initialize('--mode=cpu')

HOOMD-blue 2.4.2 DOUBLE HPMC_MIXED TBB SSE SSE2 
Compiled: 12/21/2018
Copyright 2009-2018 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----


In [3]:
snapshot = hoomd.data.make_snapshot(N=10,
                                   box=hoomd.data.boxdim(Lx=10, Ly=2., Lz=2.),
                                   particle_types=['A', 'B'],
                                   bond_types=['polymer'])

In [4]:
dir(snapshot)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_broadcast',
 '_broadcast_all',
 '_broadcast_box',
 '_dimensions',
 '_global_box',
 'angles',
 'bonds',
 'box',
 'broadcast',
 'broadcast_all',
 'constraints',
 'dihedrals',
 'impropers',
 'pairs',
 'particles',
 'replicate']

In [5]:
snapshot.particles.position[:] = [[-4.5, 0, 0],
                                 [-3.5, 0, 0],
                                 [-2.5, 0, 0],
                                 [-1.5, 0, 0],
                                 [-0.5, 0, 0],
                                 [0.5, 0, 0],
                                 [1.5, 0, 0],
                                 [2.5, 0, 0],
                                 [3.5, 0, 0],
                                 [4.5, 0, 0]]

In [6]:
snapshot.particles.position

array([[-4.5,  0. ,  0. ],
       [-3.5,  0. ,  0. ],
       [-2.5,  0. ,  0. ],
       [-1.5,  0. ,  0. ],
       [-0.5,  0. ,  0. ],
       [ 0.5,  0. ,  0. ],
       [ 1.5,  0. ,  0. ],
       [ 2.5,  0. ,  0. ],
       [ 3.5,  0. ,  0. ],
       [ 4.5,  0. ,  0. ]], dtype=float32)

In [7]:
# set particle types
snapshot.particles.typeid[0:6] = 0
snapshot.particles.typeid[6:] = 1


In [8]:
snapshot.bonds.resize(9)
snapshot.bonds.group[:] = [[0, 1],
                          [1, 2],
                          [2, 3],
                          [3, 4],
                          [4, 5],
                          [5, 6],
                          [6, 7],
                          [7, 8],
                          [8, 9]
                          ]

In [9]:
snapshot.replicate(1, 20, 20)

In [10]:
hoomd.init.read_snapshot(snapshot)

HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 4000 particles


In [11]:
hoomd.dump.gsd(filename='init_config.gsd', group=hoomd.group.all(), period=None)

In [12]:
#force field!
nl = hoomd.md.nlist.cell()
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)

In [13]:
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
lj.pair_coeff.set('A', 'B', epsilon=0.1, sigma=1.3)
lj.pair_coeff.set('B', 'B', epsilon=1.0, sigma=1.3)

In [14]:
bonds = hoomd.md.bond.harmonic()

In [15]:
bonds.bond_coeff.set('polymer', r0=1.0, k=50.)

In [16]:
#set integration method
hoomd.md.integrate.mode_standard(dt=0.005)


In [17]:
help(hoomd.md.integrate.nvt)

Help on class nvt in module hoomd.md.integrate:

class nvt(hoomd.integrate._integration_method)
 |  NVT Integration via the Nosé-Hoover thermostat.
 |  
 |  Args:
 |      group (:py:mod:`hoomd.group`): Group of particles on which to apply this method.
 |      kT (:py:mod:`hoomd.variant` or :py:obj:`float`): Temperature set point for the Nosé-Hoover thermostat. (in energy units).
 |      tau (float): Coupling constant for the Nosé-Hoover thermostat. (in time units).
 |  
 |  :py:class:`nvt` performs constant volume, constant temperature simulations using the Nosé-Hoover thermostat,
 |  using the MTK equations described in Refs. `G. J. Martyna, D. J. Tobias, M. L. Klein  1994 <http://dx.doi.org/10.1063/1.467468>`_ and
 |  `J. Cao, G. J. Martyna 1996 <http://dx.doi.org/10.1063/1.470959>`_.
 |  
 |  :py:class:`nvt` is an integration method. It must be used in connection with :py:class:`mode_standard`.
 |  
 |  :py:class:`nvt` uses the proper number of degrees of freedom to compute the temp

In [18]:
all_group = hoomd.group.all()
integrator = hoomd.md.integrate.nvt(group=all_group, kT=0.4, tau=50.)
integrator.randomize_velocities(seed=42)

In [19]:
hoomd.analyze.log(filename='output.log',
                 quantities=['potential_energy', 'temperature'],
                 period=500,
                 overwrite=True)

In [20]:
hoomd.dump.gsd('trajectory.gsd', period=1e3, group=all_group, overwrite=True)

In [21]:
hoomd.run(1e4)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 800
notice(2): Particles with 2 exclusions             : 3200
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 2687 / 10000 | TPS 268.389 | ETA 00:00:27
Time 00:00:20 | Step 5495 / 10000 | TPS 280.759 | ETA 00:00:16
Time 00:00:30 | Step 8143 / 10000 | TPS 264.756 | ETA 00:00:07
Time 00:00:37 | Step 10000 / 10000 | TPS 245.848 | ETA 00:00:00
Average TPS: 266.175
---------
-- Neighborlist stats:
1035 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 80 / n_neigh_avg: 25.3113
shortest rebuild period: 6
-- Cell list stats:
Dimension: 3, 13, 13
n_min    : 0 / n_max: 34 / n_avg: 7.88955
** run complete **
